In [ ]:
# Step 1: Read the data
# Step 2: Separate Input(Eng) and Output(France) Senetnces
# Step 3: Tokenize and Padding (Intput and output separetely)
# Step 4: Word Embeddings using glove
# Step 5: Creating the Model


In [ ]:
NUM_SENTENCES = 20000
MAX_NUM_WORDS = 20000
count = 0
input_sentences = []
output_sentences = []
output_sentences_inputs = []


In [ ]:
with open(r"C:\MyWork\MyLearning\Career Growth\ML\Files\DataSet\fra-eng.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

In [ ]:
for line in lines:
    
    count += 1
    
    if count > NUM_SENTENCES:
        break
        
    input_sentence,output,_ = line.rstrip().split('\t')
    
    output_sentence = output + ' <eos>'
    output_sentence_input = '<sos> ' + output
    
    input_sentences.append(input_sentence)
    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)

In [ ]:
len(input_sentences)

In [ ]:
len(output_sentences)

In [ ]:
len(output_sentences_inputs)

In [ ]:
input_sentences[:4]

In [ ]:
output_sentences[:4]

In [ ]:
output_sentences_inputs[:4]

In [ ]:
# Tokenization and Padding for input_sentences

from keras.preprocessing.text import Tokenizer 

input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)

input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index

max_input_len = max(len(sen) for sen in input_integer_seq)

max_input_len

In [ ]:
input_integer_seq[:10]

In [ ]:
[x for x in word2idx_inputs.items()][:10]

In [ ]:
max_input_len

In [ ]:
# Tokenization and Padding for output_sentences 
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)
word2idx_outputs = output_tokenizer.word_index


In [ ]:
output_integer_seq[:10]

In [ ]:
output_input_integer_seq[:10]

In [ ]:
[x for x in word2idx_outputs.items()][:10]

In [ ]:
num_words_output = len(word2idx_outputs) + 1

num_words_output

In [ ]:
max_out_len = max(len(sen) for sen in output_integer_seq)

max_out_len

In [ ]:
# Padding Input text Sequences

from keras.preprocessing.sequence import pad_sequences

encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len) # max_input_len =5

encoder_input_sequences[:10]

In [ ]:
encoder_input_sequences.shape

In [ ]:
decoder_output_sequences  = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post') # =12

decoder_output_sequences[:5]

In [ ]:
decoder_output_sequences.shape

In [ ]:
# Word Embeddings

from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open(r'C:\MyWork\MyLearning\Career Growth\ML\Files\DataSet\glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')    
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [ ]:
# 3523 unique words in the input.
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)

num_words

In [ ]:
EMBEDDING_SIZE = 100
embedding_matrix = zeros((num_words, EMBEDDING_SIZE))

In [ ]:
embedding_matrix.shape

In [ ]:
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
from keras.layers import Input, LSTM, GRU, Dense, Embedding

embedding_layer = Embedding(num_words,EMBEDDING_SIZE,weights=[embedding_matrix],input_length=max_input_len)
# Embedding(3453,100,weights=[embedding_matrix],input_length= 5)

In [ ]:
import numpy as np

decoder_targets_one_hot = np.zeros((len(input_sentences),max_out_len,num_words_output),dtype='float32')

In [ ]:
decoder_targets_one_hot

In [ ]:
decoder_targets_one_hot.shape


In [ ]:
for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [ ]:
decoder_targets_one_hot

In [ ]:
decoder_output_sequences

In [ ]:
# Create Encoder and Decoder

LSTM_NODES= 256

In [ ]:
encoder_inputs_placeholder = Input(shape=(max_input_len,)) # max_input_len
x = embedding_layer(encoder_inputs_placeholder) # embedding_layer : Glove embedding layer
encoder = LSTM(LSTM_NODES, return_state=True)

encoder_outputs, h, c = encoder(x)
encoder_states = [h, c]

In [ ]:
decoder_inputs_placeholder = Input(shape=(max_out_len,))

decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)
decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)


decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
from keras.models import Model

model = Model([encoder_inputs_placeholder,decoder_inputs_placeholder], decoder_outputs)

model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
r = model.fit([encoder_input_sequences, decoder_input_sequences],
              decoder_targets_one_hot,batch_size=10,
              epochs=20,validation_split=0.1,)